In [1]:
import requests
import zipfile
import numpy as np
import pandas as pd
import os

url = "https://opendart.fss.or.kr/api/corpCode.xml"
api_key = "b328110a98b1c7a1cf44a732ba4a9bb4c1f133df"
params = {'crtfc_key':api_key}
response = requests.get(url, params = params)

In [2]:
os.chdir('T:\\index\\95_곽용하\\운용\\코드')

In [3]:
# save zip file in local
with open('./id.zip','wb') as fp:
    fp.write(response.content)

# extract file
zf = zipfile.ZipFile('./id.zip')
zf.extractall()

In [4]:
xml_path = os.path.abspath('./CORPCODE.xml')

import xml.etree.ElementTree as ET
tree = ET.parse(xml_path)
root = tree.getroot()
tags_list = root.findall('list')

# DataFrame
def convert(tag: ET.Element) -> dict:
    conv = {}
    for child in list(tag):
        conv[child.tag] = child.text
    return conv

tags_list_dict = [convert(x) for x in tags_list]
df = pd.DataFrame(tags_list_dict)
df.replace(' ', np.NaN, inplace=True)

In [5]:
def find_code(df, target):
    return (df[df['corp_name']==target]['corp_code'])

In [6]:
stk_names = pd.read_csv("T:\\index\\95_곽용하\\연구\\0_CA\\IPO\\stk_names_ksp_20231204.csv",header=0, encoding='cp949')
stk_names = stk_names['STK_NM'].to_list()

In [7]:
stk_list = []
for i in range(len(stk_names)):
    stk = stk_names[i]
    names = find_code(df, stk)
    stk_list = np.append(stk_list, names)

In [8]:
stk_list[0]

'00119195'

### functions

In [9]:
# 증권신고서 주요정보 - 지분증권 OpenAPI 호출
def API_info(stock, api_key = api_key):
    url = "https://opendart.fss.or.kr/api/accnutAdtorNmNdAdtOpinion.json"
    params = {
        "crtfc_key": api_key,
        "corp_code": stock,
        "bsns_year": "2022",
        "reprt_code": "11011"
    }
    response = requests.get(url, params=params)
    return response

In [10]:
# 그룹별로 나누기
def by_group(data):
    json = data.json()
    group_list = []
    for group in json.get('group'):
        title = group.get('title')
        tag_list = group.get('list')
        group_list.append({'title': title, 'list': tag_list})
    return group_list

In [70]:
i = 2
stkk = stk_list[i]
response = API_info(stkk)
a = response.json()

stk_name = a.get('list')[0]['corp_name']
rcept_no = a.get('list')[0]['rcept_no']
aud_opn  = a.get('list')[0]['adt_opinion']
emphs    = a.get('list')[0]['emphs_matter']

KeyError: 'emphs_matter'

In [11]:
#imsi = pd.DataFrame(columns=['STK_NM','RCEPT','AUD','EMPHS'])
imsi = []

for i in range(len(stk_list)):
    stkk = stk_list[i]
    response = API_info(stkk)
    a = response.json()
    
    try:
        stk_name = a.get('list')[0]['corp_name']
        rcept_no = a.get('list')[0]['rcept_no']
        aud_opn  = a.get('list')[0]['adt_opinion']
        emphs    = a.get('list')[0]['emphs_matter']
        
    except TypeError:
        stk_name = np.NaN
        rcept_no = np.NaN
        aud_opn = np.NaN
        emphs = np.NaN        
    
    except KeyError:
        stk_name = np.NaN
        rcept_no = np.NaN
        aud_opn = np.NaN
        emphs = np.NaN
    b = [stk_name, rcept_no, aud_opn, emphs]
    print(b)
    imsi = np.append(imsi,b)
    

[nan, nan, nan, nan]
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['삼양홀딩스', '20230316001063', '적정', '해당사항없음']
['하이트진로', '20230316001349', '적정', '해당사항없음']
['유한양행', '20230315001431', '적정', '해당사항 없음']
['하이트진로홀딩스', '20230316001297', '적정', '특기사항 없음']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['DL', '20230728000644', '적정', '- 연결재무제표 재작성\n- 재무제표 재작성']
['유유제약', '20230322001027', '적정', '해당사항 없음']
['일동홀딩스', '20230316001183', '적정', '우리의 의견으로는 별첨된 회사의 재무제표는 회사의 2022년 12월 31일 및 2021년 12월 31일 현재의 재무상태와 동일로 종료되는 양 보고기간의 재무성과 및 현금흐름을 한국채택국제회계기준에 따라, 중요성의 관점에서 공정하게 표시하고 있습니다.']
['한국앤컴퍼니', '20230324001090', '적정', '해당사항 없음']
[nan, nan, nan, nan]
['기아', '20230309000734', '적 정', '해당사항 없음']
[nan, nan, nan, nan]
['대유플러스', '20230324000003', '적정', '1) 계속기업가정의 불확실성']
['노루홀딩스', '20230316001033', '적정', '해당사항 없음']
['한화손해보험', '20230314001218', '적정의견', '강조사항 없음']
['삼화페인트공업', '20230316001290', '적정', '해당사항 없음']
['롯데손해보험', '20230323001495', '적정', 'COVID-19 불확실성 증대에 따른\n감사보고서 이용자의 주의를 요하는 사항 ']
[nan, nan, nan, nan]
[nan, n

['유화증권', '20230323001499', '적정', '해당사항없음']
['유안타증권', '20230314000094', '적정', '없음']
['한진중공업홀딩스', '20230323001291', '적정', '해당사항 없음']
['대한항공', '20230314000740', '적정', '해당사항 없음']
['영진약품', '20230315001174', '적정', '우리의 의견으로는 별첨된 회사의 재무제표는 회사의 2022년 12월 31일과 2021년 12월 31일 현재의 재무상태, 동일로 종료되는 양 보고기간의 재무성과 및 현금흐름을 한국채택국제회계 기준에 따라 중요성의 관점에서 공정하게 표시하고 있습니다.']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['대신증권', '20230316001545', '적정', '회사의 2021년 12월 31일로 종료되는 보고기간의 재무제표는 삼일회계법인이 감사하였으며, 이 감사인의 2022년 3월 10일자 감사보고서에는 적정의견이 표명되었습니다. 한편 해당 감사인이 적정의견을 표명한 재무제표는 재무제표에 대한 주석3 (27) 3)에서 기술되어 있는 회계정책 변경사항을 반영하기 전의 재무제표이며, 비교표시된 2021년 12월 31일로 종료되는 재무제표는 해당 회계정책 변경사항이 반영된 재무제표입니다. 변경사항으로 인하여 2021년 12월 31일 현재 회사의 순자산 및 동일로 종료되는 보고기간의 당기순이익에 미치는 효과는 없습니다.']
['LG', '20230321001235', '적정', '(별도) 해당사항 없음\n(연결) COVID19의 영향의 불확실성']
['SNT다이내믹스', '20230220001625', '적정', '해당사항없음']
[nan, nan, nan, nan]
['방림', '20221230000430', '적 정', '해당사항없음']
['KG모빌리티', '20230314001136', '적정', '(1) 회생절차 종결(2) 상장폐지사유 발생 및 개선계획 이행여부 심의 요

['남선알미늄', '20230322000952', '적정(연결,별도)', '수주산업']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['일정실업', '20230322001314', '한정', '1. 중요 왜곡표시위험을 식별하고 평가\n2. 재고자산평가, 충당부채\n3. 종속회사 투자주식\n4. 특수관계자 거래\n5. 수익인식 및 채권채무']
['윌비스', '20230321001489', '적정', '해당사항없음']
[nan, nan, nan, nan]
['율촌화학', '20230328000998', '적정', '해당사항 없음']
['호텔신라', '20230308000580', '적정 (감사의견)', '해당사항 없음']
['금비', '20221215000437', '적정', '특이사항 없음']
['한미사이언스', '20230321001527', '적정', '해당사항 없음']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['경인전자', '20230315000359', '적정', '해당사항 없음']
['삼성전기', '20230307000653', '적정', '해당사항 없음']
['SIMPAC', '20230320000844', '적정', '해당사항 없음']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['무림페이퍼', '20230320000891', '적정', '"해당사항 없음"']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['신원', '20230324000944', '적정의견', '해당사항 없음']
[nan, nan, nan, nan]
['광동제약', '20230321001353', '적정의견', '해당사항 없음']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
[

[nan, nan, nan, nan]
[nan, nan, nan, nan]
['인지컨트롤스', '20230324001000', '적정', '해당사항없음']
['인팩', '20230322000936', '적정', '해당사항 없음']
[nan, nan, nan, nan]
['WISCOM', '20230316000445', '적정', '해당사항없음']
[nan, nan, nan, nan]
['디씨엠', '20230315000573', '적정', '"해당사항 없음"']
['한국콜마홀딩스', '20230321001245', '적정', '해당사항 없음']
[nan, nan, nan, nan]
['대원화성', '20230316001139', '적정', '없음']
[nan, nan, nan, nan]
['KPX케미칼', '20230316000950', '적정', '해당사항 없음']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['미래산업', '20230320000617', '적정', '해당사항 없음']
['제이준코스메틱', '20230322001335', '적정', '(1) 주요자산 취득에 대한 검토']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['남해화학', '20230316001430', '적 정', '해당 사항 없음']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['부국철강', '20230323000263', '적정', '지적사항 없음']
[nan, nan, nan, nan]
[nan, nan, nan, nan]
['동서', '20230316000463', '적정', '해당사항 없음']
[nan, nan, nan, nan]
['BGF', '20230320000850', '적정', '해당사항 없음']
['마니커', '202303

SSLError: HTTPSConnectionPool(host='opendart.fss.or.kr', port=443): Max retries exceeded with url: /api/accnutAdtorNmNdAdtOpinion.json?crtfc_key=b328110a98b1c7a1cf44a732ba4a9bb4c1f133df&corp_code=01012206&bsns_year=2022&reprt_code=11011 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1125)')))

In [12]:
imsi

array(['nan', 'nan', 'nan', ..., 'nan', 'nan', 'nan'], dtype='<U688')

In [14]:
len(imsi)/4

613.0

In [15]:
imsi.reshape(613,4)

array([['nan', 'nan', 'nan', 'nan'],
       ['nan', 'nan', 'nan', 'nan'],
       ['nan', 'nan', 'nan', 'nan'],
       ...,
       ['nan', 'nan', 'nan', 'nan'],
       ['팬오션', '20230321001130', '적정', '해당사항 없음'],
       ['nan', 'nan', 'nan', 'nan']], dtype='<U688')

In [18]:
dd = pd.DataFrame(imsi.reshape(613,4))
dd.columns = ['STK_NM','RCEPT','AUD','EMPHS']
dd_ = dd.dropna()

In [24]:
dd_[dd_['EMPHS'].str.contains('계속기업')]

,STK_NM,RCEPT,AUD,EMPHS
16,대유플러스,20230324000003,적정,1) 계속기업가정의 불확실성
57,페이퍼코리아,20230323001248,적정,계속기업 관련\n중요한 불확실성
65,국보,20230814002776,적정,계속기업 관련 중요한 불확실성\n\n[연결감사보고서]\n연결재무제표에 대한 주석 4...
66,금호전기,20230321001127,적정,계속기업 관련 중요한 불확실성
273,부산주공,20230323001492,적정,계속기업 관련 중요한 불확실성
278,한창,20230330001135,적정,계속기업 관련 중요한 불확실성
328,대구백화점,20230412001808,적정,-계속기업 가정의 중요한 불확실성\n보고기간 손실 발생 및 장기차입금의 유동성 대체...
339,AK홀딩스,20230323001504,적정,부문의 계속기업가정의 불확실성
441,삼화전자공업,20231017000217,적정,계속기업 관련 중요한 불확실성
458,STX,20230323001138,적정,"감사의견에는 영향을 미치지 않는 사항으로서,\n이용자는 계속기업 관련 중요한 불확실..."


In [23]:
dd.to_csv("T:\\index\\95_곽용하\\연구\\0_CA\\연말_감사인의견\\dart_20231204.csv", encoding='cp949')

In [25]:
dd_[dd_['EMPHS'].str.contains('계속기업')].to_csv("T:\\index\\95_곽용하\\연구\\0_CA\\연말_감사인의견\\dart_example.csv", encoding='cp949')